In [1]:
import os
import time as T
import torch
from IPython.display import clear_output
from bindsnet.network.monitors import Monitor
from bindsnet.encoding import poisson
from bindsnet.models import DiehlAndCook2015
from bindsnet.preprocessing import NumentaPreprocessor

In [2]:
try:
    os.mkdir('logs')
except:
    pass

In [3]:
def logs(start_time,name, msg):
    lfname = f'logs/{start_time}_{os.path.basename(name)[:-4]}_spikes.csv'
    with open(lfname, 'a') as f:
        f.write(msg)

In [4]:
datapath = './dataset/'
encpath = './encoding/'
datafiles = [os.path.join(datapath, x) for x in os.listdir(datapath)]
encfiles = [os.path.join(encpath, x[:-4])+'.enc' for x in os.listdir(datapath)]

In [5]:
enc = NumentaPreprocessor()

In [6]:
encodings = [enc.process(csvfile=d, use_cache=True, cachedfile=e)
             for d, e in zip(datafiles, encfiles)]

In [7]:
tracklist = sorted(zip(encfiles, encodings))

In [8]:
time = 500
network = DiehlAndCook2015(5000, dt=1.0, norm=48.95, inh=3)
exc_monitor = Monitor(network.layers['Ae'], ['v', 's'], time=time)
network.add_monitor(exc_monitor, name='exc')

In [9]:
def relax(network, time=50):
    img = torch.zeros(5000)
    inpts = {'X': poisson(img, time)}
    network.run(inpts=inpts, time=time)

In [ ]:
start = int(T.time())
for name, track in tracklist:
    logs(start, name, 'Iteration,Neuron,Spikes\n')
    for i in range(4, len(track)):
        print(f'Track {os.path.basename(name)[:-4]}   Iteration {i-3}')
        orig = 255*torch.cat((track[i-4], track[i-3], track[i-2], track[i-1], track[i]))
        pt = poisson(orig, time)
        
        inpts = {'X': pt}
        network.run(inpts=inpts, time=time)
        relax(network)
        
        spikes = exc_monitor.get('s')
        voltage = exc_monitor.get('v')
        
        for neuron, value in enumerate(torch.sum(spikes, dim=1)):
            logs(start, name, f'{i-4},{neuron},{int(value)}\n')
        
        clear_output(wait=True)
network.save(f'trained_{start}.net')

Track track01   Iteration 137
